In [7]:
from mintalib import functions as fx



class PricesMethods:
    _prices = None

    def atr(self, period: int = 14):
        kwds = dict(period=period)
        return fx.ATR(self._prices, **kwds)



class SeriesMethods:
    _series = None

    def ema(self, period: int = 20):
        kwds = dict(period=period)
        return fx.EMA(self._series, **kwds)

    def sma(self, period: int = 20):
        kwds = dict(period=period)
        return fx.SMA(self._series, **kwds)


def register_pandas(name="ta"):
    import pandas as pd

    @pd.api.extensions.register_dataframe_accessor(name)
    class PricesAccessor(PricesMethods, SeriesMethods):
        def __init__(self, prices):
            self._validate(prices)
            self._prices = prices

        @staticmethod
        def _validate(prices):
            if "close" not in prices.columns:
                raise AttributeError("Prices must have a 'close' column") 

        @property
        def _series(self):
            return self._prices["close"]


    @pd.api.extensions.register_series_accessor(name)
    class SeriesAccessor(SeriesMethods):
        def __init__(self, series):
            self._series = series


register_pandas()


/var/folders/c3/54m9yxns31905dkkhzl115lw0000gn/T/ipykernel_22172/2578004761.py:30: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.PricesAccessor'> under name 'ta' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PricesAccessor(PricesMethods, SeriesMethods):
/var/folders/c3/54m9yxns31905dkkhzl115lw0000gn/T/ipykernel_22172/2578004761.py:46: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.SeriesAccessor'> under name 'ta' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  class SeriesAccessor(SeriesMethods):


In [8]:
import yfinance as yf
from mintalib.indicators import SMA

# fetch prices (eg with yfinance)
prices = yf.Ticker('AAPL').history('5y')

# convert column and index names to lower case
prices = prices.rename(columns=str.lower).rename_axis(index=str.lower)

# compute and append indicators to prices
result = prices.assign(
    sma50 = SMA(50),
)
result

,open,high,low,close,volume,dividends,stock splits,sma50
date,,,,,,,,
2019-12-19 00:00:00-05:00,67.742584,68.149765,67.609283,67.868614,98369200,0.0,0.0,NaN
2019-12-20 00:00:00-05:00,68.404266,68.506058,67.514763,67.728050,275978000,0.0,0.0,NaN
2019-12-23 00:00:00-05:00,67.992226,68.893845,67.953445,68.833252,98572000,0.0,0.0,NaN
2019-12-24 00:00:00-05:00,69.000496,69.048973,68.571502,68.898697,48478800,0.0,0.0,NaN
2019-12-26 00:00:00-05:00,69.031998,70.282631,69.002914,70.265663,93121200,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
2024-12-13 00:00:00-05:00,247.820007,249.289993,246.240005,248.130005,33155300,0.0,0.0,232.215428
2024-12-16 00:00:00-05:00,247.990005,251.380005,247.649994,251.039993,51694800,0.0,0.0,232.705213
2024-12-17 00:00:00-05:00,250.080002,253.830002,249.779999,253.479996,51356400,0.0,0.0,233.345886


In [9]:
prices.ta.atr()

date
2019-12-19 00:00:00-05:00         NaN
2019-12-20 00:00:00-05:00         NaN
2019-12-23 00:00:00-05:00         NaN
2019-12-24 00:00:00-05:00         NaN
2019-12-26 00:00:00-05:00         NaN
                               ...   
2024-12-13 00:00:00-05:00    3.560010
2024-12-16 00:00:00-05:00    3.572153
2024-12-17 00:00:00-05:00    3.606285
2024-12-18 00:00:00-05:00    3.815836
2024-12-19 00:00:00-05:00    3.823641
Length: 1259, dtype: float64

In [10]:
prices.close.ta.sma()

date
2019-12-19 00:00:00-05:00           NaN
2019-12-20 00:00:00-05:00           NaN
2019-12-23 00:00:00-05:00           NaN
2019-12-24 00:00:00-05:00           NaN
2019-12-26 00:00:00-05:00           NaN
                                ...    
2024-12-13 00:00:00-05:00    237.855499
2024-12-16 00:00:00-05:00    239.157499
2024-12-17 00:00:00-05:00    240.430499
2024-12-18 00:00:00-05:00    241.418999
2024-12-19 00:00:00-05:00    242.456049
Length: 1259, dtype: float64

In [11]:
prices.ta.sma(50)

date
2019-12-19 00:00:00-05:00           NaN
2019-12-20 00:00:00-05:00           NaN
2019-12-23 00:00:00-05:00           NaN
2019-12-24 00:00:00-05:00           NaN
2019-12-26 00:00:00-05:00           NaN
                                ...    
2024-12-13 00:00:00-05:00    232.215428
2024-12-16 00:00:00-05:00    232.705213
2024-12-17 00:00:00-05:00    233.345886
2024-12-18 00:00:00-05:00    233.796448
2024-12-19 00:00:00-05:00    234.205514
Length: 1259, dtype: float64

In [12]:
prices.ta.ema()

date
2019-12-19 00:00:00-05:00           NaN
2019-12-20 00:00:00-05:00           NaN
2019-12-23 00:00:00-05:00           NaN
2019-12-24 00:00:00-05:00           NaN
2019-12-26 00:00:00-05:00           NaN
                                ...    
2024-12-13 00:00:00-05:00    239.859602
2024-12-16 00:00:00-05:00    240.924402
2024-12-17 00:00:00-05:00    242.120172
2024-12-18 00:00:00-05:00    242.684918
2024-12-19 00:00:00-05:00    243.356926
Length: 1259, dtype: float64